In [5]:
# STANDALONE TO DOWNLOAD MOVIES GIVEN CSV FILE

import csv
import time
import os
import requests
import numpy as np
import pandas as pd
import urllib

#########################################################
'''
BASE STUFF THAT IS ALSO DEFINED ON TOP
'''
def requestResults(url):
    r = requests.get(BASE_URL + url + "&api_key=" + API_KEY)
    return r.json()

# Constants
BASE_URL = "https://api.themoviedb.org/3/"
API_KEY = "9767d17413ec9d9729c2cca238df02da"
GENRE_MAP = {}
for g in requestResults("genre/movie/list?x=1")[u'genres']:
    GENRE_MAP[g['id']] = g['name']
    
#########################################################

# Download one image to filepath, note that imgpath is prepended with a /
def _downloadImageToFile(imgpath, filename):
    # Does not return anything
    urllib.urlretrieve("https://image.tmdb.org/t/p/w500" + imgpath, filename)
    
def downloadImagesToFolder(csv_file, foldername): 
    # Folder name without any trailing /; If folder does not exist, create it
    if not os.path.exists(foldername):
        os.makedirs(foldername)

    df = pd.read_csv(csv_file)
    df = np.array(df[['tmdb_id', 'poster_path']])
    
    # TMDB limits 4 requests per second
    hit = 3 # Once hit reaches 0, call timer and reset hit to 3
    count = 0
    
    for row in df:
        mid = str(row[0])
        poster_path = row[1]
        count += 1
        if type(poster_path) != str:
            continue
        if poster_path.strip() == '': # If empty path, continue
            continue
        _downloadImageToFile(imgpath=poster_path, filename=foldername + "/" + mid + ".jpg")
        hit -= 1
        if hit <= 0:
            hit = 3
            time.sleep(1)
        if count % 200 == 0:
            print('Done downloading %d posters' % count)

In [6]:
downloadImagesToFolder(csv_file = '../data/movie-posters-list.csv', foldername='posters')

IOError: [Errno socket error] [Errno 54] Connection reset by peer